In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import math
import seaborn as sns
import csv

In [ ]:
games = pd.read_excel('../Results.xlsx', sheet_name='Games')
turns = pd.read_excel('../Results.xlsx', sheet_name='Turns')
pvp_games = games.query('GameState == "END" and idPerson1 not in [10, 30, 203, 217]'
                        ' and idPerson2 not in [10, 30, 203, 217]')
pvp_games_ids = pvp_games['idGame']